In [28]:
import pandas as pd
import plotly.express as px
import folium
from haversine import haversine

df = pd.read_csv('/home/thiagoferes/projects/CDS/FTC/dataset/train.csv')
df1 = df.copy()

#removendo linhas 'NaN' do Festival
df1 = df1.loc[df1['Festival'] != 'NaN ', :]

#removendo linhas 'NaN' do Road_traffic_density
df1 = df1.loc[df1['Road_traffic_density'] != 'NaN', :]

#removendo linhas 'NaN' do City
df1 = df1.loc[df1['City'] != 'NaN ', :]

# convertendo Delivery_person_Age para int
linhas_selecionadas = df1['Delivery_person_Age'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()
df1['Delivery_person_Age'] = df1['Delivery_person_Age'].astype(int)

# convertendo Delivery_person_Ratings para float
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype(float)

# convertendo Order_Date para datetime
df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], format=('%d-%m-%Y'))

# convertendo multiple_deliveries para int
linhas_selecionadas = df1['multiple_deliveries'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()
df1['multiple_deliveries'] = df1['multiple_deliveries'].astype(int)

# removendo espacos dentro de strings
df1.loc[:, 'ID'] = df1.loc[:, 'ID'].str.strip()
df1.loc[:, 'Road_traffic_density'] = df1.loc[:, 'Road_traffic_density'].str.strip()
df1.loc[:, 'Type_of_order'] = df1.loc[:, 'Type_of_order'].str.strip()
df1.loc[:, 'Type_of_vehicle'] = df1.loc[:, 'Type_of_vehicle'].str.strip()
df1.loc[:, 'City'] = df1.loc[:, 'City'].str.strip()

#limpando a coluna de time taken
df1['Time_taken(min)'] = df1['Time_taken(min)'].apply( lambda x: x.split('(min) ')[1])
df1['Time_taken(min)'] = df1['Time_taken(min)'].astype(int)

In [29]:
df1['City'].unique()

array(['Urban', 'Metropolitian', 'Semi-Urban'], dtype=object)

## Visao Empresa:


### 1. Quantidade de pedidos por dia.


In [30]:
cols = ['ID', 'Order_Date']
df_aux = df1.loc[:, cols].groupby(['Order_Date']).count().reset_index()

px.bar(df_aux, x='Order_Date', y='ID')



### 2. Quantidade de pedidos por semana.


In [31]:
df1['week_of_year'] = df1['Order_Date'].dt.strftime('%U')

df_aux = df1.loc[:, ['week_of_year', 'Order_Date']].groupby(['week_of_year']).count().reset_index()
px.line(df_aux, x='week_of_year', y='Order_Date')


### 3. Distribuição dos pedidos por tipo de tráfego.


In [32]:
df_aux = df1.loc[:, ['ID', 'Road_traffic_density']].groupby(['Road_traffic_density']).count().reset_index()
df_aux['delivery_perc'] = df_aux['ID']/df_aux['ID'].sum()
px.pie(df_aux, values='delivery_perc', names='Road_traffic_density')

### 4. Comparação do volume de pedidos por cidade e tipo de tráfego.


In [33]:
df_aux = df1.loc[:, ['ID', 'City', 'Road_traffic_density']].groupby(['City', 'Road_traffic_density']).count().reset_index()
px.scatter(df_aux, x='City', y='Road_traffic_density', size='ID', color='City')

### 5. A quantidade de pedidos por entregador por semana.

In [34]:
df_aux1 = df1.loc[:, ['week_of_year', 'ID']].groupby(['week_of_year']).count().reset_index()
df_aux2 = df1.loc[:, ['week_of_year', 'Delivery_person_ID']].groupby(['week_of_year']).nunique().reset_index()

df_aux = pd.merge( df_aux1, df_aux2, how='inner')
df_aux['order_by_deliver'] = df_aux['ID'] / df_aux['Delivery_person_ID']
px.line(df_aux, x='week_of_year', y='order_by_deliver')

### 6. A localização central de cada cidade por tipo de tráfego.

In [35]:
df_aux = df1.loc[:, ['City', 'Road_traffic_density', 'Delivery_location_latitude', 'Delivery_location_longitude']].groupby(['City', 'Road_traffic_density']).median().reset_index()

map = folium.Map()

for index, location_info in df_aux.iterrows():
    folium.Marker([location_info['Delivery_location_latitude'], 
                   location_info['Delivery_location_longitude']], 
                   popup=location_info[['City', 'Road_traffic_density']]).add_to(map)

map

## Visao Entregador:

### 1. A menor e maior idade dos entregadores.


In [56]:
print(f'A maior idade eh:', df1.loc[:, 'Delivery_person_Age'].max())
print(f'A menor idade eh:', df1.loc[:, 'Delivery_person_Age'].min())

A maior idade eh: 50
A menor idade eh: 15


### 2. A pior e a melhor condição de veículos.


In [38]:
print(f'A melhor condicao do veiculo:', df1.loc[:, 'Vehicle_condition'].max())
print(f'A pior condicao do veiculo:', df1.loc[:, 'Vehicle_condition'].min())

A melhor condicao do veiculo: 3
A pior condicao do veiculo: 0


### 3. A avaliação média por entregador.


In [39]:
df1.loc[:, ['Delivery_person_Ratings', 'Delivery_person_ID']].groupby(['Delivery_person_ID']).mean().reset_index()

,Delivery_person_ID,Delivery_person_Ratings
0,AGRRES010DEL01,4.761538
1,AGRRES010DEL02,4.671429
2,AGRRES010DEL03,4.575000
3,AGRRES01DEL01,4.522222
4,AGRRES01DEL02,4.700000
...,...,...
1315,VADRES19DEL02,4.632727
1316,VADRES19DEL03,4.670270
1317,VADRES20DEL01,4.620370
1318,VADRES20DEL02,4.591111


### 4. A avaliação média e o desvio padrão por tipo de tráfego.


In [40]:
df_avg_std_rating_by_traffic = df1.loc[:, ['Delivery_person_Ratings', 'Road_traffic_density']].groupby(['Road_traffic_density']).agg({'Delivery_person_Ratings': ['mean', 'std']})
df_avg_std_rating_by_traffic.columns = ['delivery_mean', 'delivery_std']
df_avg_std_rating_by_traffic.reset_index()

,Road_traffic_density,delivery_mean,delivery_std
0,High,4.652230,0.273044
1,Jam,4.594019,0.329778
2,Low,4.645011,0.338080
3,Medium,4.660138,0.274245
4,NaN,3.865169,2.487199


### 5. A avaliação média e o desvio padrão por condições climáticas.


In [41]:
df_avg_std_rating_by_weather = df1.loc[:, ['Delivery_person_Ratings', 'Weatherconditions']].groupby(['Weatherconditions']).agg({'Delivery_person_Ratings': ['mean', 'std']})
df_avg_std_rating_by_weather.columns = ['delivery_mean', 'delivery_std']
df_avg_std_rating_by_weather.reset_index()

,Weatherconditions,delivery_mean,delivery_std
0,conditions Cloudy,4.651871,0.281197
1,conditions Fog,4.652965,0.275060
2,conditions NaN,3.865169,2.487199
3,conditions Sandstorms,4.611748,0.310852
4,conditions Stormy,4.611819,0.313096
5,conditions Sunny,4.654868,0.396674
6,conditions Windy,4.616128,0.304565


### 6. Os 10 entregadores mais rápidos por cidade.


In [42]:
df2 = df1.loc[:, ['Delivery_person_ID', 'Time_taken(min)', 'City']].groupby(['City', 'Delivery_person_ID']).mean().sort_values(['Time_taken(min)', 'City']).reset_index()

df_aux1 = df2.loc[df2['City'] == 'Metropolitian', :].head(10)
df_aux2 = df2.loc[df2['City'] == 'Urban', :].head(10)
df_aux3 = df2.loc[df2['City'] == 'Semi-Urban', :].head(10)

df3 = pd.concat([df_aux1, df_aux2, df_aux3]).reset_index(drop=True)

df3

,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,KNPRES03DEL02,15.750000
1,Metropolitian,ALHRES02DEL02,17.800000
2,Metropolitian,KNPRES01DEL01,19.125000
3,Metropolitian,KOLRES01DEL03,19.125000
4,Metropolitian,KOCRES02DEL02,19.250000
5,Metropolitian,KOCRES08DEL03,19.800000
6,Metropolitian,KOCRES16DEL03,20.000000
7,Metropolitian,KOCRES02DEL03,20.375000
8,Metropolitian,GOARES14DEL01,20.400000
9,Metropolitian,DEHRES20DEL03,20.428571


### 7. Os 10 entregadores mais lentos por cidade.

In [43]:
df2 = df1.loc[:, ['Delivery_person_ID', 'Time_taken(min)', 'City']].groupby(['City', 'Delivery_person_ID']).mean().sort_values(['Time_taken(min)', 'City'], ascending=False).reset_index()

df_aux1 = df2.loc[df2['City'] == 'Metropolitian', :].head(10)
df_aux2 = df2.loc[df2['City'] == 'Urban', :].head(10)
df_aux3 = df2.loc[df2['City'] == 'Semi-Urban', :].head(10)

df3 = pd.concat([df_aux1, df_aux2, df_aux3]).reset_index(drop=True)

df3

,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,AGRRES02DEL01,39.428571
1,Metropolitian,AURGRES11DEL03,38.500000
2,Metropolitian,KOLRES03DEL03,38.142857
3,Metropolitian,LUDHRES17DEL03,37.000000
4,Metropolitian,ALHRES18DEL02,36.666667
5,Metropolitian,GOARES08DEL03,36.000000
6,Metropolitian,ALHRES12DEL01,35.000000
7,Metropolitian,KNPRES06DEL01,34.555556
8,Metropolitian,ALHRES010DEL01,34.500000
9,Metropolitian,KOCRES09DEL02,34.333333


## Visao Restaurantes:

### 1. A quantidade de entregadores únicos.


In [44]:
print(f'O numero de entregadores unicos eh:', df1.loc[:, 'Delivery_person_ID'].nunique())

O numero de entregadores unicos eh: 1320


### 2. A distância média dos resturantes e dos locais de entrega.


In [45]:
cols = ['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']
df1['distance'] = df1.loc[:, cols].apply(
    lambda x: haversine(
        (x['Restaurant_latitude'], x['Restaurant_longitude']),
        (x['Delivery_location_latitude'], x['Delivery_location_longitude'])
        ), 
    axis=1
)

avg_distance = df1['distance'].mean()
print(f'A distancia media das entregas eh:',avg_distance, 'KM')


A distancia media das entregas eh: 38.50435351464306 KM


### 3. O tempo médio e o desvio padrão de entrega por cidade.


In [46]:
delivery_avg_std_by_city = df1.loc[:, ['Time_taken(min)', 'City']].groupby(['City']).agg({'Time_taken(min)': ['mean','std']})
delivery_avg_std_by_city.columns = ['avg_time', 'std_time']
delivery_avg_std_by_city.reset_index()

,City,avg_time,std_time
0,Metropolitian,27.425533,9.131709
1,Semi-Urban,49.710526,2.724992
2,Urban,23.206066,8.851762


In [47]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),week_of_year,distance
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,...,High,2,Snack,motorcycle,0,No,Urban,24,11,3.025153
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,...,Jam,2,Snack,scooter,1,No,Metropolitian,33,12,20.183558
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,...,Low,0,Drinks,motorcycle,1,No,Urban,26,11,1.552760
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,...,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21,14,7.790412
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,...,High,1,Snack,scooter,1,No,Metropolitian,30,12,6.210147


### 4. O tempo médio e o desvio padrão de entrega por cidade e tipo de pedido.


In [48]:
delivery_avg_std_by_city = df1.loc[:, ['Time_taken(min)', 'City', 'Type_of_order']].groupby(['City', 'Type_of_order']).agg({'Time_taken(min)': ['mean','std']})
delivery_avg_std_by_city.columns = ['avg_time', 'std_time']
delivery_avg_std_by_city.reset_index()

,City,Type_of_order,avg_time,std_time
0,Metropolitian,Buffet,27.305076,9.153779
1,Metropolitian,Drinks,27.315351,9.038785
2,Metropolitian,Meal,27.607623,9.215682
3,Metropolitian,Snack,27.468536,9.114292
4,Semi-Urban,Buffet,49.707317,2.731702
5,Semi-Urban,Drinks,49.625000,2.459347
6,Semi-Urban,Meal,50.300000,3.041665
7,Semi-Urban,Snack,49.408163,2.707385
8,Urban,Buffet,23.559880,9.052978
9,Urban,Drinks,23.306856,8.922801


### 5. O tempo médio e o desvio padrão de entrega por cidade e tipo de tráfego.


In [49]:
delivery_avg_std_by_city = df1.loc[:, ['Time_taken(min)', 'City', 'Road_traffic_density']].groupby(['City', 'Road_traffic_density']).agg({'Time_taken(min)': ['mean','std']})
delivery_avg_std_by_city.columns = ['avg_time', 'std_time']
delivery_avg_std_by_city.reset_index()

,City,Road_traffic_density,avg_time,std_time
0,Metropolitian,High,28.140898,7.904645
1,Metropolitian,Jam,31.976991,9.476203
2,Metropolitian,Low,22.257675,6.794772
3,Metropolitian,Medium,27.729966,8.308064
4,Metropolitian,NaN,26.156250,8.227362
5,Semi-Urban,High,50.125000,2.629956
6,Semi-Urban,Jam,49.841270,2.717095
7,Semi-Urban,Medium,47.400000,2.011080
8,Urban,High,24.305335,8.494842
9,Urban,Jam,27.993164,10.078271


### 6. O tempo médio de entrega durantes os Festivais.

In [50]:
delivery_avg_std_by_city = df1.loc[:, ['Time_taken(min)', 'Festival']].groupby(['Festival']).agg({'Time_taken(min)': ['mean','std']})
delivery_avg_std_by_city.columns = ['avg_time', 'std_time']
delivery_avg_std_by_city = delivery_avg_std_by_city.reset_index()

linhas_selecionadas = delivery_avg_std_by_city['Festival'] == 'Yes '
delivery_avg_std_by_city = delivery_avg_std_by_city.loc[linhas_selecionadas, :]
delivery_avg_std_by_city

,Festival,avg_time,std_time
1,Yes,45.51976,4.001862
